In [1]:
from collections import defaultdict
import json
import numpy as np
import pandas as pd
# from docx import Document


import json
import os
from copy import deepcopy
import time
import ast
import re
import requests

In [2]:
from openai import OpenAI

# Get the first key from the uploaded dictionary
env_file_key = "openai.env"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

# # Load the content into a variable
# env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_content
# openai.api_key = api_key

client = OpenAI(
    # # This is the default and can be omitted
    api_key=api_key,
)


In [3]:
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt
    )
    return response.choices[0].message.content.strip()

### Prepare data

In [8]:
data_file = "data/new_data/unique_risks_model_cards_v3.json"
data = pd.read_json(data_file)

In [9]:
data.head(2)

,modelId,author,creation_time,last_modified,downloads,likes,library_name,tags,pipeline_tag,card_data_language,card_data_license,card_data_library_name,card_data_tags,card_data_base_model,card_data_datasets,model_card_metadata,model_card,risks_limitations_bias,risk_section_len
2,facebook/fasttext-language-identification,facebook,2023-03-06 12:52:50,2023-06-09T12:39:43+00:00,53352708,135,fasttext,"[fasttext, text-classification, language-ident...",text-classification,None,cc-by-nc-4.0,fasttext,"[text-classification, language-identification]",None,None,"{'base_model': None, 'datasets': None, 'eval_r...",\n# fastText (Language Identification)\n\nfast...,### Limitations and bias\n\nEven if the traini...,786
4,google-bert/bert-base-uncased,google-bert,2022-03-02 23:29:04,2024-02-19T11:06:12+00:00,44931654,1678,transformers,"[transformers, pytorch, tf, jax, rust, coreml,...",fill-mask,en,apache-2.0,None,[exbert],None,"[bookcorpus, wikipedia]","{'base_model': None, 'datasets': ['bookcorpus'...",\n# BERT base model (uncased)\n\nPretrained mo...,### Limitations and bias\n\nEven if the traini...,1783


In [10]:
data.shape

(2672, 19)

In [11]:
unique_risks = data.drop_duplicates(subset="risks_limitations_bias")
print (unique_risks.shape)

(2672, 19)


In [71]:
unique_risks.head(2)

,modelId,author,creation_time,last_modified,downloads,likes,library_name,tags,pipeline_tag,card_data_language,card_data_license,card_data_library_name,card_data_tags,card_data_base_model,card_data_datasets,model_card_metadata,model_card,risks_limitations_bias,risk_section_len
2,facebook/fasttext-language-identification,facebook,2023-03-06 12:52:50,2023-06-09T12:39:43+00:00,53352708,135,fasttext,"[fasttext, text-classification, language-ident...",text-classification,None,cc-by-nc-4.0,fasttext,"[text-classification, language-identification]",None,None,"{'base_model': None, 'datasets': None, 'eval_r...",\n# fastText (Language Identification)\n\nfast...,### Limitations and bias\n\nEven if the traini...,786
4,google-bert/bert-base-uncased,google-bert,2022-03-02 23:29:04,2024-02-19T11:06:12+00:00,44931654,1678,transformers,"[transformers, pytorch, tf, jax, rust, coreml,...",fill-mask,en,apache-2.0,None,[exbert],None,"[bookcorpus, wikipedia]","{'base_model': None, 'datasets': ['bookcorpus'...",\n# BERT base model (uncased)\n\nPretrained mo...,### Limitations and bias\n\nEven if the traini...,1783


### Prompt

In [31]:
MESSAGES = [ { 'role': 'system', 'content': """
Consider the following definitions: 
    1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. 
    2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an event occurring and the severity of the consequences that would result. Risk is usually expressed in terms of risk sources, potential events, their consequences and their likelihood. 
""" },

{ 'role': 'user', 'content': """You are provided in input with cardID and a section written by an AI developer from Risks, Bias and Limitations section of an AI model card:
cardID: "{}", section: "{}".

Tasks:

1) Identify Relevance: Determine for each section whether it discusses the AI risks and potential solutions to mitigate them. Provide a response as Yes, No, or Unclear. If the answer is Yes, continue with the next steps; else, you will output empty "Risks and Mitigations" in the output JSON.

2) Analyze: Identify  all the subtexts in the section that can be categorized into - "RISKS",  "MITIGATIONS", or "MIX OF BOTH". Classify each such subtext into one of the provided three types. This subtext must match the exact sentences in the input section and is used as a reference.

3) Split: For the each reference, identify unique risks and mitigations. Classify each into one of the two types "RISKS" or "MITIGATIONS".
It is very important that each risk or mitigation identified be formatted like this: Verb + Object + [Explanation], and describes one unique risk/mitigation, is concise, consisting of one clear, to-the-point sentence, with up to maximum of 20 words. Specifically, start a risk with an action verb in active present tense (e.g., undermines, discriminates, infringes, reduces, increases etc., but NOT potentially) followed by the object and the reason in case it is not obvious and requires an explanation. Start a mitigation with verb in present tense, base form (e.g., perform, inform, investigate). Also, we want these descriptions to be read by broad public who does not have a deep knowledge about AI technology. Do not change the meaning of the concrete risk. For example, if it says the model does not works in a particular setting then you say it underperforms and not discriminates.
This would NOT be a good output: "Potentially undermines the right to privacy if the facial recognition data is not properly secured." while this is good: "Undermines the right to privacy if the facial recognition data is not properly secured.". This would NOT be a good output: "Affects all fined tuned versions of the model." as it does not specify what the risk is; instead "Transfers all the <specific risks from reference like bias, unrepresentative data> to the inherited models" is better. "Evaluations surface potential risks in use cases." is a bad output, instead specify the potential risks from the reference.
Other examples of well-formatted risks:
    * Underperforms on non-English languages 
    * Discriminates against certain players, such as women or those from certain ethnic backgrounds.
    * Undermines the safety and security when model is deployed without thorough in-domain testing.
Examples of well-formatted mitigations:
    * Perform thorough in-domain testing
    * Update model card for models using pretrained models
    * Inform indirect users when the content they're working with is created by the LLM.

Output Format: Ensure your output strictly follows this JSON structure.

{{
    "cardID": "<Card ID>",
    "section": {{
        "about risks, bias or mitigations": ["Yes"/"No"/"Unclear"],
        "risks_and_mitigations": 
    [
        {{
            "reference_text": "Reference subtext 1 from <section>",
            "type": ["RISKS", "MITIGATIONS", "MIX OF BOTH"],
            "splits": [
                {{
                "unique_risk_mitigation": "Unique risk/mitigation 1 in identified reference",
                "type": ["RISKS", "MITIGATIONS"]
                }},
                ...
                {{
                "unique_risk_mitigation": "Unique risk/mitigation N in identified reference",
                "Type": ["RISKS", "MITIGATIONS"]
                }},
            ]
        }},
        ...
        {{
            "reference_text": "Reference subtext N from <section>",
            "type": ["RISKS", "MITIGATIONS", "MIX OF BOTH"],
            "splits": [
                {{
                "unique_risk_mitigation": "Unique risk/mitigation 1 in identified reference",
                "type": ["RISKS", "MITIGATIONS"]
                }},
                ...
                {{
                "unique_risk_mitigation": "Unique risk/mitigation N in identified reference",
                "type": ["RISKS", "MITIGATIONS"]
                }},
            ]
        }},
        ...
    ]
    }}
}}


Important Notes: Do not report your reasoning steps or any preamble like 'Here is the output', ONLY the JSON result. In scenarios where there are no sentences mentioned, provide an empty JSON array for those sections.

*** Double Check your output that it contains only the requested JSON and nothing else. *** """

} ]

In [32]:
def format_prompt(MESSAGES, cardID, section): 
    messages = deepcopy(MESSAGES) 
    messages[1]['content'] = messages[1]['content'].format(cardID, section) 
    return messages

In [66]:
unique_risks.iloc[785:786]

,modelId,author,creation_time,last_modified,downloads,likes,library_name,tags,pipeline_tag,card_data_language,card_data_license,card_data_library_name,card_data_tags,card_data_base_model,card_data_datasets,model_card_metadata,model_card,risks_limitations_bias,risk_section_len
15688,M4-ai/Hercules-Mini-1.8B,M4-ai,2024-03-29 00:06:01,2024-03-30T15:18:49+00:00,504,6,transformers,"[transformers, safetensors, qwen2, text-genera...",text-generation,[en],other,transformers,None,None,[Locutusque/hercules-v4.0],"{'base_model': None, 'datasets': ['Locutusque/...",\n# Hercules-Mini-1.8B \n\n<!-- Provide a quic...,"## Bias, Risks, and Limitations\n\n<!-- This s...",583


In [64]:
messages = format_prompt(MESSAGES, "15688", unique_risks["risks_limitations_bias"].iloc[785])

In [65]:
messages

[{'role': 'system',
  'content': "\nConsider the following definitions: \n    1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. \n    2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an ev

In [67]:
response = chat_gpt(messages)

In [68]:
response

'{\n    "cardID": "15688",\n    "section": {\n        "about risks, bias or mitigations": ["Yes"],\n        "risks_and_mitigations": [\n            {\n                "reference_text": "The eos token was not setup properly, so to prevent infinite generation you\'ll need to implement a stopping criteria when the model generates the'

In [36]:
response

'{\n    "cardID": "2",\n    "section": {\n        "about risks, bias or mitigations": ["Yes"],\n        "risks_and_mitigations": [\n            {\n                "reference_text": "Any deployed use case of the model - whether commercial or not - is currently out of scope. Non-deployed use cases such as image search in a constrained environment, are also not recommended unless there is thorough in-domain testing of the model with a specific, fixed class taxonomy. This is because our safety assessment demonstrated a high need for task specific testing especially given the variability of CLIP’s performance with different class taxonomies. This makes untested and unconstrained deployment of the model in any use case currently potentially harmful.",\n                "type": ["MIX OF BOTH"],\n                "splits": [\n                    {\n                        "unique_risk_mitigation": "Undermines the safety and security when model is deployed without thorough in-domain testing.",\n 

In [25]:
response

'```json\n{\n    "cardID": "2",\n    "section": {\n        "about risks, bias or mitigations": "Yes",\n        "risks_and_mitigations": [\n            {\n                "reference_text": "Even if the training data used for this model could be characterized as fairly neutral, this model can have biased predictions:",\n                "type": "RISKS",\n                "splits": [\n                    {\n                        "unique_risk_mitigation": "Produces biased predictions despite neutral training data",\n                        "type": "RISKS"\n                    }\n                ]\n            },\n            {\n                "reference_text": "[Example predictions showing bias]",\n                "type": "RISKS",\n                "splits": [\n                    {\n                        "unique_risk_mitigation": "Exhibits gender bias in job roles predictions",\n                        "type": "RISKS"\n                    },\n                    {\n                     

### Batch the data

In [37]:
def batch_data(data=data):

    tasks = []

    for i, el in data.iterrows():
        userID = i 
        # if (len(chosen_ids) and userID in chosen_ids) or (not len(chosen_ids)):
        
        section = el["risks_limitations_bias"]

        messages = format_prompt(MESSAGES, userID, section)

        task = {
        "custom_id": f"task-{int(userID)}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4o",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": messages,
            }
        }
    
        tasks.append(task)

    return tasks

In [38]:
# Creating the file

file_name = "batch_risks.jsonl"

tasks = batch_data(data=unique_risks)
with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

### Upload batch file

In [39]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [40]:
batch_file

FileObject(id='file-pu0CxC6ZYA6H7eU6IE8jjFJt', bytes=18714957, created_at=1721771322, filename='batch_risks.jsonl', object='file', purpose='batch', status='processed', status_details=None)

### Create batch job

In [41]:
# Creating the batch job
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [42]:
batch_job.id

'batch_9AhgAKcRCNJpYZQYBOQUipBe'

In [46]:

# Checking batch status
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_9AhgAKcRCNJpYZQYBOQUipBe', completion_window='24h', created_at=1721771345, endpoint='/v1/chat/completions', input_file_id='file-pu0CxC6ZYA6H7eU6IE8jjFJt', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1721772241, error_file_id=None, errors=None, expired_at=None, expires_at=1721857745, failed_at=None, finalizing_at=1721772102, in_progress_at=1721771347, metadata=None, output_file_id='file-IBKWsO9FrIAdjfs8exfGXwyq', request_counts=BatchRequestCounts(completed=2672, failed=0, total=2672))


### Retrieve results

In [47]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [48]:
result_file_name = "data/new_data/llm_analysis/LLM_risk_analysis_results_gppt4o.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [49]:
import json
import ast

In [50]:
result_file_name = "data/new_data/llm_analysis/LLM_risk_analysis_results_gppt4o.jsonl"
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [24]:
for res in results:
    print (res)
    break

{'id': 'batch_req_5v7sbal5tOW52tLVh7Jryhhv', 'custom_id': 'task-0', 'response': {'status_code': 200, 'request_id': '7f8d8fa761a125ce444669d0e2674e1d', 'body': {'id': 'chatcmpl-9bbHdgi9kfoobiQRoc6cMYUKBLH9b', 'object': 'chat.completion', 'created': 1718748481, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n    "cardID": "0",\n    "section": {\n        "about risks, bias or mitigations": "Yes",\n        "risks_and_mitigations": [\n            {\n                "reference_text": "Even if the training data used for this model could be characterized as fairly neutral, this model can have biased predictions:",\n                "type": "RISKS",\n                "splits": [\n                    {\n                        "unique_risk_mitigation": "Produces biased predictions despite neutral training data.",\n                        "type": "RISKS"\n                    }\n                ]\n            },\n            {\n                "

In [51]:
FULL_RES = []
cnt_errors = []

# Reading only the first results
for res in results:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    try:
        res = ast.literal_eval(result)
        # print (index)
        res["Incident ID"] = int(index)
        FULL_RES.append(res)
    except Exception as e:
        print (e)
        print (result)
        cnt_errors.append(index)
        continue

EOL while scanning string literal (<unknown>, line 7)
{
    "cardID": "15688",
    "section": {
        "about risks, bias or mitigations": "Yes",
        "risks_and_mitigations": [
            {
                "reference_text": "The eos token was not setup properly, so to prevent infinite generation you'll need to implement a stopping criteria when the model generates the 


In [52]:
cnt_errors

['15688']

In [69]:
###############################
# save result
with open(f"data/new_data/llm_analysis/LLM_risk_analysis_results_gppt4o.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation

### Analyse Results

In [3]:
extracted_risks = pd.read_json("data/new_data/llm_analysis/LLM_risk_analysis_results_gppt4o.json")
# process them to contain clean strings
risk_ref_list = []
mit_ref_list = []
risk_list = []
mit_list = []

for i, row in extracted_risks.iterrows():
    risk_sentences = []
    mit_sentences = []
    risks = []
    mits = []
    if "risks_and_mitigations" in row["section"].keys():
        for sentence in row["section"]['risks_and_mitigations']:
            if sentence["type"] == "RISKS" or sentence["type"] == ["RISKS"]:
                risk_sentences += sentence["reference_text"] + ' ** '
            if sentence["type"] == "MITIGATIONS" or sentence["type"] == ["MITIGATIONS"]:
                mit_sentences += sentence["reference_text"] + ' ** '

            if "splits" in sentence.keys():
                for split in sentence["splits"]:
                    if split["type"] == "RISKS" or split["type"] == ["RISKS"]:
                        risks.append(split["unique_risk_mitigation"])
                    if split["type"] == "MITIGATIONS" or split["type"] == ["MITIGATIONS"]:
                        mits.append(split["unique_risk_mitigation"])

    risk_ref_list.append(risk_sentences)
    mit_ref_list.append(mit_sentences)
    risk_list.append(risks)
    mit_list.append(mits)


In [4]:

#  assign it to the dataframe
extracted_risks["risk_reference"] = risk_ref_list
extracted_risks["mitigation_reference"] = mit_ref_list
extracted_risks["risks"] = risk_list
extracted_risks["mitigations"] = mit_list

In [5]:
extracted_risks

,cardID,section,Incident ID,risk_reference,mitigation_reference,risks,mitigations
0,2,"{'about risks, bias or mitigations': 'Yes', 'r...",2,Even if the training data used for this model ...,,Produces biased predictions despite neutral tr...,
1,4,"{'about risks, bias or mitigations': 'Yes', 'r...",4,Even if the training data used for this model ...,,Produces biased predictions despite neutral tr...,
2,5,"{'about risks, bias or mitigations': 'Yes', 'r...",5,Certain use cases which would fall under the d...,,Undermines safety and security when model is d...,Perform thorough in-domain testing. ** Perform...
3,10,"{'about risks, bias or mitigations': 'Yes', 'r...",10,Certain use cases which would fall under the d...,,Undermines safety when deployed without thorou...,Perform thorough in-domain testing with a spec...
4,11,"{'about risks, bias or mitigations': 'Yes', 'r...",11,Even if the training data used for this model ...,,Produces biased predictions despite neutral tr...,
...,...,...,...,...,...,...,...
2666,395164,"{'about risks, bias or mitigations': 'Yes', 'r...",395164,"The model, while powerful, may have limitation...",Users should verify the model's recommendation...,Underperforms in understanding highly speciali...,Verify the model's recommendations with curren...
2667,397247,"{'about risks, bias or mitigations': 'Yes', 'r...",397247,The model was not trained to be factual or tru...,,Underperforms in generating factual or true re...,Implement additional safety mechanisms and con...
2668,397368,"{'about risks, bias or mitigations': 'No', 'ri...",397368,,,,
2669,397779,"{'about risks, bias or mitigations': 'Yes', 'r...",397779,,Users should exercise caution and perform addi...,Underperforms in scenarios outside specific ge...,Perform additional testing for specific use ca...


In [6]:
extracted_risks.drop(["Incident ID"], axis=1, inplace=True)

In [7]:
extracted_risks.head(2)

,cardID,section,risk_reference,mitigation_reference,risks,mitigations
0,2,"{'about risks, bias or mitigations': 'Yes', 'r...",Even if the training data used for this model ...,,Produces biased predictions despite neutral tr...,
1,4,"{'about risks, bias or mitigations': 'Yes', 'r...",Even if the training data used for this model ...,,Produces biased predictions despite neutral tr...,


In [2]:
extracted_risks = pd.read_json("data/new_data/llm_analysis/LLM_risk_analysis_results_gppt4o.json")
sentences = []
split_sentences = []
types = []
split_types = []
for i,row in extracted_risks.iterrows():
    if "risks_and_mitigations" in row["section"].keys():
        for sentence in row["section"]['risks_and_mitigations']:
            sentences.append(sentence["reference_text"])
            if type(sentence["type"]) != str : types.append(sentence["type"][0])
            else: types.append(sentence["type"])
            if "splits" in sentence.keys():
                for split in sentence["splits"]:
                    split_sentences.append(split["unique_risk_mitigation"])
                    if type(split["type"]) != str : split_types.append(split["type"][0])
                    else: split_types.append(split["type"])

In [3]:
df = pd.DataFrame({"sentences": sentences, "types": types})
df_split = pd.DataFrame({"sentences": split_sentences, "types": split_types})

In [4]:
df_split.head()

,sentences,types
0,Produces biased predictions despite neutral tr...,RISKS
1,Produces biased predictions despite neutral tr...,RISKS
2,Transfers bias to all fine-tuned versions,RISKS
3,Undermines safety and security when model is d...,RISKS
4,Perform thorough in-domain testing.,MITIGATIONS


In [5]:
df.groupby(['types']).count()

,sentences
types,
MITIGATIONS,1462
MIX OF BOTH,1753
RISKS,4847


In [6]:
df_split.groupby(['types']).count()

,sentences
types,
MITIGATIONS,3951
RISKS,8926


In [7]:
a = df.groupby(['sentences']).count().sort_values(by='types', ascending=False)
print (a)

                                                    types
sentences                                                
Users (both direct and downstream) should be ma...    124
This bias will also affect all fine-tuned versi...     46
The model was not trained to be factual or true...     44
Faces and people in general may not be generate...     37
Testing conducted to date has been in English, ...     37
...                                                   ...
Limited Domain: The model's effectiveness may b...      1
Limited Domain Expertise: While knowledgeable a...      1
Limited Context: The model makes predictions ba...      1
Limitations: The model's performance depends on...      1
한편, 본 언어모델은 주로 한국어 텍스트로 학습되었으며, 이들의 특성을 전이할 수 있...      1

[5356 rows x 1 columns]


In [8]:
b = df_split.groupby(['sentences']).count().sort_values(by='types', ascending=False)
print (b)

                                                    types
sentences                                                
Underperforms on non-English languages                 95
Carries risks with use                                 49
Produces factually incorrect output                    45
Inform users about the risks, biases, and limit...     38
Underperforms in non-English languages                 36
...                                                   ...
Consider using additional validation steps to e...      1
Consult a licensed medical professional before ...      1
Inform end-users about limitations and possible...      1
Consult a qualified financial professional for ...      1
Überprüfen Sie die Ausgaben.                            1

[8153 rows x 1 columns]


In [9]:
risks_df = df_split[df_split['types'] == 'RISKS']
c = risks_df.groupby(['sentences']).count().sort_values(by='types', ascending=False)
print (c)

                                                    types
sentences                                                
Underperforms on non-English languages                 95
Carries risks with use                                 49
Produces factually incorrect output                    45
Fails to cover all scenarios                           36
Underperforms in non-English languages                 36
...                                                   ...
Hallucinates on questions about code commands           1
Hallucinates on questions about Verus technolog...      1
Hallucinates on advanced questions                      1
Hallucinates new ships, prices, features not in...      1
Yields inaccurate results for unrelated classif...      1

[5396 rows x 1 columns]


In [10]:
c.to_csv("data/new_data/unique_risks_count.csv")

In [16]:
df.shape

(8062, 2)

In [17]:
df_split.shape

(12877, 2)

In [18]:
df.drop_duplicates(subset="sentences", keep="first", inplace=True)
df_split.drop_duplicates(subset="sentences", keep="first", inplace=True)

In [19]:
df.shape

(5356, 2)

In [20]:
df_split.shape

(8153, 2)

In [21]:
df.to_csv("data/new_data/unique_risks_mitigations_gpto.csv")
df_split.to_csv("data/new_data/unique_risks_mitigation_splits_gpto.csv")

In [22]:
df.groupby(['types']).count()

,sentences
types,
MITIGATIONS,948
MIX OF BOTH,1182
RISKS,3226


In [23]:
df_split.groupby(['types']).count()

,sentences
types,
MITIGATIONS,2759
RISKS,5394
